### Imports

In [165]:
import os
from PIL import Image

### Helper functions

Function to apply histogram equalization on an image

In [166]:
def EqualizeImage(image):
    histogram = CalculateHistogram(image);
    cumulative_histogram = CalculateCumulativeHistogram(histogram)
    new_intensities = CalculateEqualizedHistogram(cumulative_histogram, image.size)
    equalized_image = CalculateEqualizedImage(image, new_intensities)
    return equalized_image

### Main functions

CalculateHistogram function

In [167]:
def CalculateHistogram(image):
    n, m = image.size
    histogram = [0] * 256
    for i in range(n):
        for j in range(m):
            histogram[image.getpixel((i,j))] += 1
    return histogram

CalculateCumulativeHistogram function

In [168]:
def CalculateCumulativeHistogram(histogram):
    cumulative_histogram = [0] * 256
    cumulative_histogram[0] = histogram[0]
    for i in range(1, len(histogram)):
        cumulative_histogram[i] = cumulative_histogram[i-1] + histogram[i]
    return cumulative_histogram

CalculateEqualizedHistogram function

In [169]:
def CalculateEqualizedHistogram(cumulative_histogram, image_size):
    n, m = image_size
    new_intensities = []
    for i in range(len(cumulative_histogram)):
        new_value = round(255 / ((n*m) - cumulative_histogram[0]) * (cumulative_histogram[i] - cumulative_histogram[0]))
        new_intensities.append(new_value)
    return new_intensities

CalculateEqualizedImage function

In [170]:
def CalculateEqualizedImage(image,  new_intensities):
    n, m = image.size
    equalized_image = Image.new("L", (n,m))
    for i in range(n):
        for j in range(m):
            old_value = image.getpixel((i,j))
            new_value = new_intensities[old_value]
            equalized_image.putpixel((i,j), new_value)
    return equalized_image

SegmentOptimalThresholding function

In [171]:
def SegmentOptimalThresholding(image):
    n, m = image.size
    segmented_image = Image.new("1", (n,m), 1)
    segmented_image.putpixel((0,0), 0)
    segmented_image.putpixel((0,m-1), 0)
    segmented_image.putpixel((n-1,0), 0)
    segmented_image.putpixel((n-1,m-1), 0)
    old_threshold = -1
    while True:
        sum_background = 0
        num_background = 0
        sum_object = 0
        num_object = 0
        for i in range(n):
            for j in range(m):
                if segmented_image.getpixel((i,j)) == 0:
                    sum_background += image.getpixel((i,j))
                    num_background += 1
                else:
                    sum_object += image.getpixel((i,j))
                    num_object += 1
        mean_background = sum_background / num_background
        mean_object = sum_object / num_object
        new_threshold = (mean_background + mean_object) / 2
        if new_threshold == old_threshold:
            break
        for i in range(n):
            for j in range(m):
                if image.getpixel((i,j)) < new_threshold:
                    segmented_image.putpixel((i,j), 0)
                else:
                    segmented_image.putpixel((i,j), 1)
        old_threshold = new_threshold
    return segmented_image

### Test

In [181]:
current_directory = os.getcwd()
image_path = current_directory + R"\images\1.jpg"
image = Image.open(image_path)
image = image.convert("L")

In [182]:
segmented_image = SegmentOptimalThresholding(image)
equalized_image = EqualizeImage(image)
segmented_equalized_image = SegmentOptimalThresholding(equalized_image)

In [183]:
image.save(current_directory + R"\1_original.jpg")
segmented_image.save(current_directory + R"\1_segmented.jpg")
equalized_image.save(current_directory + R"\1_equalized.jpg")
segmented_equalized_image.save(current_directory + R"\1_segmented_equalized.jpg")

### Comments